In [1]:
import argparse
import os
import pathlib
import sys

import duckdb
import pandas as pd
from notebook_init_utils import bandicoot_check, init_notebook
from pycytominer import aggregate, feature_select

root_dir, in_notebook = init_notebook()

profile_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot/NF1_organoid_data")).resolve(),
    root_dir,
)

In [2]:
patient_ids_path = pathlib.Path(f"{profile_base_dir}/data/patient_IDs.txt").resolve(
    strict=True
)
patients = pd.read_csv(patient_ids_path, header=None, names=["patient_id"], dtype=str)[
    "patient_id"
].to_list()

all_patients_output_path = pathlib.Path(
    f"{profile_base_dir}/data/all_patient_profiles"
).resolve()
all_patients_output_path.mkdir(parents=True, exist_ok=True)

In [3]:
levels_to_merge_dict = {
    "sc": [],
    "organoid": [],
}

In [4]:
for patient in patients:
    norm_path = pathlib.Path(
        f"{profile_base_dir}/data/{patient}/image_based_profiles/4.normalized_profiles"
    )
    for file in norm_path.glob("*.parquet"):
        if "sc" in file.name:
            levels_to_merge_dict["sc"].append(file)
        elif "organoid" in file.name:
            levels_to_merge_dict["organoid"].append(file)

In [5]:
feature_select_ops = [
    "drop_na_columns",
    "blocklist",
    # "variance_threshold", # comment out to remove variance thresholding
    # "correlation_threshold", # comment out to remove correlation thresholding
]
metadata_cols = [
    x
    for x in pd.read_parquet(levels_to_merge_dict["sc"][0]).columns
    if "Metadata_" in x
]
na_cutoff = 0.05
corr_threshold = 0.9
freq_cut = 0.01
unique_cut = 0.01

In [6]:
for compartment, files in levels_to_merge_dict.items():
    print(f"Found {len(files)} files for {compartment} level.")
    list_of_dfs = []
    for file in files:
        patient_id = str(file.parent).split("/")[-3]
        df = pd.read_parquet(file)
        df["Metadata_patient_tumor"] = patient_id
        list_of_dfs.append(df)
    df = pd.concat(list_of_dfs, ignore_index=True)

    print(f"Concatenated DataFrame for {compartment} has the shape: {df.shape}")
    df.to_parquet(
        f"{all_patients_output_path}/{compartment}_profiles.parquet",
        index=False,
    )
    if compartment == "sc":
        blocklist_path = pathlib.Path(
            f"{root_dir}/4.processing_image_based_profiles/data/blocklist/sc_blocklist.txt"
        )
        metadata_cols = [
            "Metadata_patient_tumor",
            "Metadata_object_id",
            "Metadata_dose_unit",
            "Metadata_dose",
            "Metadata_treatment",
            "Metadata_target",
            "Metadata_class",
            "Metadata_therapeutic_categories",
            "Metadata_image_set",
            "Metadata_Well",
            "Metadata_parent_organoid",
            "Area.Size.Shape_Cell_CENTER.X",
            "Area.Size.Shape_Cell_CENTER.Y",
            "Area.Size.Shape_Cell_CENTER.Z",
        ]
        # only perform feature selection on DMSO and staurosporine treatments and apply to rest of profiles
        all_trt_df = df.copy()
        df = df.loc[df["Metadata_treatment"].isin(["DMSO", "Staurosporine"])]
        # feature selection
        feature_columns = [col for col in df.columns if col not in metadata_cols]
        features_df = df[feature_columns]
        fs_profiles = feature_select(
            features_df,
            operation=feature_select_ops,
            features=feature_columns,
            blocklist_file=blocklist_path,
            na_cutoff=na_cutoff,
            # corr_threshold=corr_threshold, # comment out to use default value
            # freq_cut=freq_cut, # comment out to use default value
            # unique_cut=unique_cut, # comment out to use default value
        )
        original_data_shape = features_df.shape
        # apply feature selection to all profiles
        fs_profiles = all_trt_df[
            [col for col in all_trt_df.columns if col in fs_profiles.columns]
        ]
        fs_profiles = pd.concat(
            [
                all_trt_df[metadata_cols].reset_index(drop=True),
                fs_profiles.reset_index(drop=True),
            ],
            axis=1,
        )
        fs_profiles.to_parquet(
            f"{all_patients_output_path}/sc_fs_profiles.parquet",
            index=False,
        )
        feature_columns = [
            col for col in fs_profiles.columns if col not in metadata_cols
        ]
        features_df = fs_profiles[feature_columns]
        # aggregate the profiles
        sc_agg_df = aggregate(
            population_df=fs_profiles,
            strata=[
                "Metadata_patient_tumor",
                "Metadata_Well",
                "Metadata_treatment",
                "Metadata_dose",
                "Metadata_dose_unit",
                "Metadata_target",
                "Metadata_class",
                "Metadata_therapeutic_categories",
            ],
            features=feature_columns,
            operation="median",
        )
        sc_agg_df.to_parquet(
            f"{all_patients_output_path}/sc_agg_profiles.parquet",
            index=False,
        )
        # consensus profiles
        sc_consensus_df = aggregate(
            population_df=fs_profiles,
            strata=[
                "Metadata_patient_tumor",
                "Metadata_treatment",
                "Metadata_dose",
                "Metadata_dose_unit",
                "Metadata_target",
                "Metadata_class",
                "Metadata_therapeutic_categories",
            ],
            features=feature_columns,
            operation="median",
        )
        sc_consensus_df.to_parquet(
            f"{all_patients_output_path}/sc_consensus_profiles.parquet",
            index=False,
        )
        print("The number features before feature selection:", original_data_shape[1])
        print("The number features after feature selection:", fs_profiles.shape[1])

    elif compartment == "organoid":
        blocklist_path = pathlib.Path(
            f"{root_dir}/4.processing_image_based_profiles/data/blocklist/organoid_blocklist.txt"
        )
        metadata_cols = [
            "Metadata_patient_tumor",
            "Metadata_object_id",
            "Metadata_dose_unit",
            "Metadata_dose",
            "Metadata_treatment",
            "Metadata_target",
            "Metadata_class",
            "Metadata_therapeutic_categories",
            "Metadata_image_set",
            "Metadata_Well",
            "Metadata_single_cell_count",
            "Area.Size.Shape_Organoid_CENTER.X",
            "Area.Size.Shape_Organoid_CENTER.Y",
            "Area.Size.Shape_Organoid_CENTER.Z",
        ]
        all_trt_df = df.copy()
        df = df.loc[df["Metadata_treatment"].isin(["DMSO", "Staurosporine"])]
        feature_columns = [col for col in df.columns if col not in metadata_cols]
        features_df = df[feature_columns]
        fs_profiles = feature_select(
            features_df,
            operation=feature_select_ops,
            features=feature_columns,
            blocklist_file=blocklist_path,
            na_cutoff=na_cutoff,
            corr_threshold=corr_threshold,
            freq_cut=freq_cut,
            unique_cut=unique_cut,
        )
        fs_profiles = all_trt_df[
            [col for col in all_trt_df.columns if col in fs_profiles.columns]
        ]
        original_data_shape = features_df.shape
        fs_profiles = pd.concat(
            [
                all_trt_df[metadata_cols].reset_index(drop=True),
                fs_profiles.reset_index(drop=True),
            ],
            axis=1,
        )
        fs_profiles.to_parquet(
            f"{all_patients_output_path}/organoid_fs_profiles.parquet",
            index=False,
        )
        feature_columns = [
            col for col in fs_profiles.columns if col not in metadata_cols
        ]
        features_df = fs_profiles[feature_columns]
        # aggregate the profiles
        agg_df = aggregate(
            population_df=fs_profiles,
            strata=[
                "Metadata_patient_tumor",
                "Metadata_Well",
                "Metadata_treatment",
                "Metadata_dose",
                "Metadata_dose_unit",
                "Metadata_target",
                "Metadata_class",
                "Metadata_therapeutic_categories",
            ],
            features=feature_columns,
            operation="median",
        )
        agg_df.to_parquet(
            f"{all_patients_output_path}/organoid_agg_profiles.parquet",
            index=False,
        )
        # consensus profiles
        consensus_df = aggregate(
            population_df=fs_profiles,
            strata=[
                "Metadata_patient_tumor",
                "Metadata_Well",
                "Metadata_treatment",
                "Metadata_dose",
                "Metadata_dose_unit",
                "Metadata_target",
                "Metadata_class",
                "Metadata_therapeutic_categories",
            ],
            features=feature_columns,
            operation="median",
        )
        consensus_df.to_parquet(
            f"{all_patients_output_path}/organoid_consensus_profiles.parquet",
            index=False,
        )

        print("The number features before feature selection:", original_data_shape[1])
        print("The number features after feature selection:", fs_profiles.shape[1])

Found 12 files for sc level.


Concatenated DataFrame for sc has the shape: (78848, 5991)
The number features before feature selection: 5977
The number features after feature selection: 5901
Found 12 files for organoid level.
Concatenated DataFrame for organoid has the shape: (32947, 2008)
The number features before feature selection: 1994
The number features after feature selection: 1973


In [7]:
import pathlib

import pandas as pd

df = pd.read_parquet(
    pathlib.Path(
        "/home/lippincm/mnt/bandicoot/NF1_organoid_data/data/all_patient_profiles/organoid_fs_profiles.parquet"
    )
)
pd.options.display.max_columns = 100
df.head()

,Metadata_patient_tumor,Metadata_object_id,Metadata_dose_unit,Metadata_dose,Metadata_treatment,Metadata_target,Metadata_class,Metadata_therapeutic_categories,Metadata_image_set,Metadata_Well,Metadata_single_cell_count,Area.Size.Shape_Organoid_CENTER.X,Area.Size.Shape_Organoid_CENTER.Y,Area.Size.Shape_Organoid_CENTER.Z,Metadata_cqc_nan_detected,Metadata_cqc_small_organoid_outlier,Metadata_cqc_large_organoid_outlier,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Colocalization_Organoid_AGP.ER_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MIN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MAX.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MIN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MAX.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEAN.K1,Colocalization_Organoid_AGP.ER_MEDIAN.K1,Colocalization_Organoid_AGP.ER_MIN.K1,Colocalization_Organoid_AGP.ER_MAX.K1,Colocalization_Organoid_AGP.ER_MEAN.K2,Colocalization_Organoid_AGP.ER_MEDIAN.K2,Colocalization_Organoid_AGP.ER_MIN.K2,Colocalization_Organoid_AGP.ER_MAX.K2,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M2,...,Texture_Organoid_AGP_Correlation_256.3,Texture_Organoid_AGP_Difference.Entropy_256.3,Texture_Organoid_AGP_Difference.Variance_256.3,Texture_Organoid_AGP_Entropy_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_AGP_Inverse.Difference.Moment_256.3,Texture_Organoid_AGP_Sum.Average_256.3,Texture_Organoid_AGP_Sum.Entropy_256.3,Texture_Organoid_AGP_Sum.Variance_256.3,Texture_Organoid_AGP_Variance_256.3,Texture_Organoid_DNA_Angular.Second.Moment_256.3,Texture_Organoid_DNA_Contrast_256.3,Texture_Organoid_DNA_Correlation_256.3,Texture_Organoid_DNA_Difference.Entropy_256.3,Texture_Organoid_DNA_Difference.Variance_256.3,Texture_Organoid_DNA_Entropy_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_DNA_Inverse.Difference.Moment_256.3,Texture_Organoid_DNA_Sum.Average_256.3,Texture_Organoid_DNA_Sum.Entropy_256.3,Texture_Organoid_DNA_Sum.Variance_256.3,Texture_Organoid_DNA_Variance_256.3,Texture_Organoid_ER_Angular.Second.Moment_256.3,Texture_Organoid_ER_Contrast_256.3,Texture_Organoid_ER_Correlation_256.3,Texture_Organoid_ER_Difference.Entropy_256.3,Texture_Organoid_ER_Difference.Variance_256.3,Texture_Organoid_ER_Entropy_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_ER_Inverse.Difference.Moment_256.3,Texture_Organoid_ER_Sum.Average_256.3,Texture_Organoid_ER_Sum.Entropy_256.3,Texture_Organoid_ER_Sum.Variance_256.3,Texture_Organoid_ER_Variance_256.3,Texture_Organoid_Mito_Angular.Second.Moment_256.3,Texture_Organoid_Mito_Contrast_256.3,Texture_Organoid_Mito_Correlation_256.3,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1

In [8]:
df.loc[df["Area.Size.Shape_Organoid_VOLUME"].isna() == True]

,Metadata_patient_tumor,Metadata_object_id,Metadata_dose_unit,Metadata_dose,Metadata_treatment,Metadata_target,Metadata_class,Metadata_therapeutic_categories,Metadata_image_set,Metadata_Well,Metadata_single_cell_count,Area.Size.Shape_Organoid_CENTER.X,Area.Size.Shape_Organoid_CENTER.Y,Area.Size.Shape_Organoid_CENTER.Z,Metadata_cqc_nan_detected,Metadata_cqc_small_organoid_outlier,Metadata_cqc_large_organoid_outlier,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Colocalization_Organoid_AGP.ER_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MIN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MAX.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MIN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MAX.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEAN.K1,Colocalization_Organoid_AGP.ER_MEDIAN.K1,Colocalization_Organoid_AGP.ER_MIN.K1,Colocalization_Organoid_AGP.ER_MAX.K1,Colocalization_Organoid_AGP.ER_MEAN.K2,Colocalization_Organoid_AGP.ER_MEDIAN.K2,Colocalization_Organoid_AGP.ER_MIN.K2,Colocalization_Organoid_AGP.ER_MAX.K2,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M2,...,Texture_Organoid_AGP_Correlation_256.3,Texture_Organoid_AGP_Difference.Entropy_256.3,Texture_Organoid_AGP_Difference.Variance_256.3,Texture_Organoid_AGP_Entropy_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_AGP_Inverse.Difference.Moment_256.3,Texture_Organoid_AGP_Sum.Average_256.3,Texture_Organoid_AGP_Sum.Entropy_256.3,Texture_Organoid_AGP_Sum.Variance_256.3,Texture_Organoid_AGP_Variance_256.3,Texture_Organoid_DNA_Angular.Second.Moment_256.3,Texture_Organoid_DNA_Contrast_256.3,Texture_Organoid_DNA_Correlation_256.3,Texture_Organoid_DNA_Difference.Entropy_256.3,Texture_Organoid_DNA_Difference.Variance_256.3,Texture_Organoid_DNA_Entropy_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_DNA_Inverse.Difference.Moment_256.3,Texture_Organoid_DNA_Sum.Average_256.3,Texture_Organoid_DNA_Sum.Entropy_256.3,Texture_Organoid_DNA_Sum.Variance_256.3,Texture_Organoid_DNA_Variance_256.3,Texture_Organoid_ER_Angular.Second.Moment_256.3,Texture_Organoid_ER_Contrast_256.3,Texture_Organoid_ER_Correlation_256.3,Texture_Organoid_ER_Difference.Entropy_256.3,Texture_Organoid_ER_Difference.Variance_256.3,Texture_Organoid_ER_Entropy_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_ER_Inverse.Difference.Moment_256.3,Texture_Organoid_ER_Sum.Average_256.3,Texture_Organoid_ER_Sum.Entropy_256.3,Texture_Organoid_ER_Sum.Variance_256.3,Texture_Organoid_ER_Variance_256.3,Texture_Organoid_Mito_Angular.Second.Moment_256.3,Texture_Organoid_Mito_Contrast_256.3,Texture_Organoid_Mito_Correlation_256.3,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1

In [9]:
df.loc[df["Metadata_single_cell_count"].isna()]

,Metadata_patient_tumor,Metadata_object_id,Metadata_dose_unit,Metadata_dose,Metadata_treatment,Metadata_target,Metadata_class,Metadata_therapeutic_categories,Metadata_image_set,Metadata_Well,Metadata_single_cell_count,Area.Size.Shape_Organoid_CENTER.X,Area.Size.Shape_Organoid_CENTER.Y,Area.Size.Shape_Organoid_CENTER.Z,Metadata_cqc_nan_detected,Metadata_cqc_small_organoid_outlier,Metadata_cqc_large_organoid_outlier,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Colocalization_Organoid_AGP.ER_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MIN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MAX.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MIN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MAX.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEAN.K1,Colocalization_Organoid_AGP.ER_MEDIAN.K1,Colocalization_Organoid_AGP.ER_MIN.K1,Colocalization_Organoid_AGP.ER_MAX.K1,Colocalization_Organoid_AGP.ER_MEAN.K2,Colocalization_Organoid_AGP.ER_MEDIAN.K2,Colocalization_Organoid_AGP.ER_MIN.K2,Colocalization_Organoid_AGP.ER_MAX.K2,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M2,...,Texture_Organoid_AGP_Correlation_256.3,Texture_Organoid_AGP_Difference.Entropy_256.3,Texture_Organoid_AGP_Difference.Variance_256.3,Texture_Organoid_AGP_Entropy_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_AGP_Inverse.Difference.Moment_256.3,Texture_Organoid_AGP_Sum.Average_256.3,Texture_Organoid_AGP_Sum.Entropy_256.3,Texture_Organoid_AGP_Sum.Variance_256.3,Texture_Organoid_AGP_Variance_256.3,Texture_Organoid_DNA_Angular.Second.Moment_256.3,Texture_Organoid_DNA_Contrast_256.3,Texture_Organoid_DNA_Correlation_256.3,Texture_Organoid_DNA_Difference.Entropy_256.3,Texture_Organoid_DNA_Difference.Variance_256.3,Texture_Organoid_DNA_Entropy_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_DNA_Inverse.Difference.Moment_256.3,Texture_Organoid_DNA_Sum.Average_256.3,Texture_Organoid_DNA_Sum.Entropy_256.3,Texture_Organoid_DNA_Sum.Variance_256.3,Texture_Organoid_DNA_Variance_256.3,Texture_Organoid_ER_Angular.Second.Moment_256.3,Texture_Organoid_ER_Contrast_256.3,Texture_Organoid_ER_Correlation_256.3,Texture_Organoid_ER_Difference.Entropy_256.3,Texture_Organoid_ER_Difference.Variance_256.3,Texture_Organoid_ER_Entropy_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_ER_Inverse.Difference.Moment_256.3,Texture_Organoid_ER_Sum.Average_256.3,Texture_Organoid_ER_Sum.Entropy_256.3,Texture_Organoid_ER_Sum.Variance_256.3,Texture_Organoid_ER_Variance_256.3,Texture_Organoid_Mito_Angular.Second.Moment_256.3,Texture_Organoid_Mito_Contrast_256.3,Texture_Organoid_Mito_Correlation_256.3,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1

In [10]:
df.loc[
    (df["Area.Size.Shape_Organoid_VOLUME"] > 1)
    & df["Metadata_single_cell_count"].isna()
]

,Metadata_patient_tumor,Metadata_object_id,Metadata_dose_unit,Metadata_dose,Metadata_treatment,Metadata_target,Metadata_class,Metadata_therapeutic_categories,Metadata_image_set,Metadata_Well,Metadata_single_cell_count,Area.Size.Shape_Organoid_CENTER.X,Area.Size.Shape_Organoid_CENTER.Y,Area.Size.Shape_Organoid_CENTER.Z,Metadata_cqc_nan_detected,Metadata_cqc_small_organoid_outlier,Metadata_cqc_large_organoid_outlier,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Colocalization_Organoid_AGP.ER_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MIN.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MAX.CORRELATION.COEFF,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.ER_MEAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEDIAN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MIN.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MAX.OVERLAP.COEFF,Colocalization_Organoid_AGP.ER_MEAN.K1,Colocalization_Organoid_AGP.ER_MEDIAN.K1,Colocalization_Organoid_AGP.ER_MIN.K1,Colocalization_Organoid_AGP.ER_MAX.K1,Colocalization_Organoid_AGP.ER_MEAN.K2,Colocalization_Organoid_AGP.ER_MEDIAN.K2,Colocalization_Organoid_AGP.ER_MIN.K2,Colocalization_Organoid_AGP.ER_MAX.K2,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MEDIAN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MIN.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MAX.MANDERS.COEFF.COSTES.M1,Colocalization_Organoid_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M2,...,Texture_Organoid_AGP_Correlation_256.3,Texture_Organoid_AGP_Difference.Entropy_256.3,Texture_Organoid_AGP_Difference.Variance_256.3,Texture_Organoid_AGP_Entropy_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_AGP_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_AGP_Inverse.Difference.Moment_256.3,Texture_Organoid_AGP_Sum.Average_256.3,Texture_Organoid_AGP_Sum.Entropy_256.3,Texture_Organoid_AGP_Sum.Variance_256.3,Texture_Organoid_AGP_Variance_256.3,Texture_Organoid_DNA_Angular.Second.Moment_256.3,Texture_Organoid_DNA_Contrast_256.3,Texture_Organoid_DNA_Correlation_256.3,Texture_Organoid_DNA_Difference.Entropy_256.3,Texture_Organoid_DNA_Difference.Variance_256.3,Texture_Organoid_DNA_Entropy_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_DNA_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_DNA_Inverse.Difference.Moment_256.3,Texture_Organoid_DNA_Sum.Average_256.3,Texture_Organoid_DNA_Sum.Entropy_256.3,Texture_Organoid_DNA_Sum.Variance_256.3,Texture_Organoid_DNA_Variance_256.3,Texture_Organoid_ER_Angular.Second.Moment_256.3,Texture_Organoid_ER_Contrast_256.3,Texture_Organoid_ER_Correlation_256.3,Texture_Organoid_ER_Difference.Entropy_256.3,Texture_Organoid_ER_Difference.Variance_256.3,Texture_Organoid_ER_Entropy_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_ER_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_ER_Inverse.Difference.Moment_256.3,Texture_Organoid_ER_Sum.Average_256.3,Texture_Organoid_ER_Sum.Entropy_256.3,Texture_Organoid_ER_Sum.Variance_256.3,Texture_Organoid_ER_Variance_256.3,Texture_Organoid_Mito_Angular.Second.Moment_256.3,Texture_Organoid_Mito_Contrast_256.3,Texture_Organoid_Mito_Correlation_256.3,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1

In [11]:
import pathlib

import pandas as pd

df = pd.read_parquet(
    pathlib.Path(
        "/home/lippincm/mnt/bandicoot/NF1_organoid_data/data/all_patient_profiles/sc_fs_profiles.parquet"
    )
)
pd.options.display.max_columns = 100
df.head()

,Metadata_patient_tumor,Metadata_object_id,Metadata_dose_unit,Metadata_dose,Metadata_treatment,Metadata_target,Metadata_class,Metadata_therapeutic_categories,Metadata_image_set,Metadata_Well,Metadata_parent_organoid,Area.Size.Shape_Cell_CENTER.X,Area.Size.Shape_Cell_CENTER.Y,Area.Size.Shape_Cell_CENTER.Z,Metadata_cqc_organoid_flagged,Metadata_cqc_nan_detected,Metadata_cqc_missing_parent_organoid,Metadata_cqc_small_nuclei_outlier,Metadata_cqc_large_nuclei_outlier,Metadata_cqc_mass_displacement_outlier,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_EXTENT,Area.Size.Shape_Nuclei_EULER.NUMBER,Area.Size.Shape_Nuclei_EQUIVALENT.DIAMETER,Area.Size.Shape_Nuclei_SURFACE.AREA,Colocalization_Nuclei_AGP.ER_MEAN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.ER_MEDIAN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.ER_MIN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.ER_MAX.CORRELATION.COEFF,Colocalization_Nuclei_AGP.ER_MEAN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.ER_MEDIAN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.ER_MIN.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.ER_MAX.MANDERS.COEFF.M1,Colocalization_Nuclei_AGP.ER_MEAN.MANDERS.COEFF.M2,Colocalization_Nuclei_AGP.ER_MEDIAN.MANDERS.COEFF.M2,Colocalization_Nuclei_AGP.ER_MIN.MANDERS.COEFF.M2,Colocalization_Nuclei_AGP.ER_MAX.MANDERS.COEFF.M2,Colocalization_Nuclei_AGP.ER_MEAN.OVERLAP.COEFF,Colocalization_Nuclei_AGP.ER_MEDIAN.OVERLAP.COEFF,Colocalization_Nuclei_AGP.ER_MIN.OVERLAP.COEFF,Colocalization_Nuclei_AGP.ER_MAX.OVERLAP.COEFF,Colocalization_Nuclei_AGP.ER_MEAN.K1,Colocalization_Nuclei_AGP.ER_MEDIAN.K1,Colocalization_Nuclei_AGP.ER_MIN.K1,Colocalization_Nuclei_AGP.ER_MAX.K1,Colocalization_Nuclei_AGP.ER_MEAN.K2,Colocalization_Nuclei_AGP.ER_MEDIAN.K2,Colocalization_Nuclei_AGP.ER_MIN.K2,Colocalization_Nuclei_AGP.ER_MAX.K2,Colocalization_Nuclei_AGP.ER_MEAN.MANDERS.COEFF.COSTES.M1,...,Texture_Cytoplasm_AGP_Correlation_256.3,Texture_Cytoplasm_AGP_Difference.Entropy_256.3,Texture_Cytoplasm_AGP_Difference.Variance_256.3,Texture_Cytoplasm_AGP_Entropy_256.3,Texture_Cytoplasm_AGP_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_AGP_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_AGP_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_AGP_Sum.Average_256.3,Texture_Cytoplasm_AGP_Sum.Entropy_256.3,Texture_Cytoplasm_AGP_Sum.Variance_256.3,Texture_Cytoplasm_AGP_Variance_256.3,Texture_Cytoplasm_DNA_Angular.Second.Moment_256.3,Texture_Cytoplasm_DNA_Contrast_256.3,Texture_Cytoplasm_DNA_Correlation_256.3,Texture_Cytoplasm_DNA_Difference.Entropy_256.3,Texture_Cytoplasm_DNA_Difference.Variance_256.3,Texture_Cytoplasm_DNA_Entropy_256.3,Texture_Cytoplasm_DNA_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_DNA_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_DNA_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_DNA_Sum.Average_256.3,Texture_Cytoplasm_DNA_Sum.Entropy_256.3,Texture_Cytoplasm_DNA_Sum.Variance_256.3,Texture_Cytoplasm_DNA_Variance_256.3,Texture_Cytoplasm_ER_Angular.Second.Moment_256.3,Texture_Cytoplasm_ER_Contrast_256.3,Texture_Cytoplasm_ER_Correlation_256.3,Texture_Cytoplasm_ER_Difference.Entropy_256.3,Texture_Cytoplasm_ER_Difference.Variance_256.3,Texture_Cytoplasm_ER_Entropy_256.3,Texture_Cytoplasm_ER_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_ER_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_ER_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_ER_Sum.Average_256.3,Texture_Cytoplasm_ER_Sum.Entropy_256.3,Texture_Cytoplasm_ER_Sum.Variance_256.3,Texture_Cytoplasm_ER_Variance_256.3,Texture_Cytoplasm_Mito_Angular.Second.Moment_256.3,Texture_Cytoplasm_Mito_Contrast_256.3,Texture_Cytoplasm_Mito_Correlation_256.3,Texture_Cytoplasm_Mito_Difference.Entropy_256.3,Texture_Cytoplasm_Mito_Difference.Variance_256.3,Texture_Cytoplasm_Mito_Entropy_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_Mito_Inverse.Difference.Moment_256.3,Te